In [1]:
lib_path = '/Users/ivan/Dropbox/Trading/TA_Parser/lib'
import sys;sys.path.insert(0, lib_path) if lib_path not in sys.path else None
from misc import get_dfs
from intrinio import get_api as get_intrinio_api

import pandas as pd
import numpy as np
from tqdm.cli import tqdm

trades_dfs, oi_dfs, top_dfs = get_dfs()
api = get_intrinio_api()
all_dfs = {}

In [ ]:
all_days = trades_dfs.date.unique()
for day_id, day in enumerate(all_days):
    day = pd.to_datetime(day).date()
    trades_df = trades_dfs[(trades_dfs['date'] == np.datetime64(day))]
    oi_df = oi_dfs[(oi_dfs['Date'] == np.datetime64(day))]
    top_df = top_dfs[(top_dfs['date'] == np.datetime64(day))]

    def check_trade(r: pd.Series) -> bool:
        if (pd.to_datetime(r['expiry_date']).date() - day).days < 15: return False
        #if (abs(r['delta']) < 0.1): return False
        if r['side'] == 0: return False

        direction = (r['pc']=='C' and r['side'] > 0) or (r.pc=='P' and r['side'] < 0)

        poi = oi_df[(oi_df['Symbol'] == r['root'])]
        if poi.empty:
            return False
        for i, oi in poi.iterrows():
            if oi['OI'] < r['size']: break

            if (pd.to_datetime(oi['Expiry']).date() - day).days > 92: continue

            wrongoi = oi['Mid'] * oi['OI']
            if oi['P/C']=='C':
                wrongoi += oi['Bid'] * oi['OI'] if direction else oi['Ask'] * oi['OI']
            else:
                wrongoi += oi['Ask'] * oi['OI'] if direction else oi['Bid'] * oi['OI']

            #print(oi, wrongoi)
            if wrongoi > r['size']: return False

        ptop = top_df[(top_df['root'] == r['root'])]
        found_t = False
        for i, top in ptop.iterrows():
            topdic = top.to_dict()
            rdic = r.to_dict()
            keys = topdic.keys() & r.keys()
            if {k:topdic[k] for k in topdic if k in keys} == {k:rdic[k] for k in rdic if k in keys}:
                found_t = True
                continue
            if top['date_time'] > r['date_time']: continue
            #if (pd.to_datetime(top['expiry_date']).date() - day).days > 92: continue
            if top['size'] < r['size']:
                return found_t
            #if top['size'] == r['size']: continue
            if top['size'] < top['open_int']: continue
            if top['side'] * r['side'] == 0: return False
            if top['side'] * r['side'] < 0 and top['pc']==r['pc']: return False
            if top['side'] * r['side'] > 0 and top['pc']!=r['pc']: return False

        return found_t

    trades_df_c = trades_df.copy()
    #trades_df_c['check_trade'] = False
    #trades_df_c['check_trade'] = trades_df_c.apply(check_trade, axis=1)
    #trades_df_c = trades_df_c[(trades_df_c['check_trade'] == True)].reset_index(drop=False)#['root'].unique()

    def check_results(r: pd.Series) -> pd.Series:
        symbol = f"{r['root']}{pd.Timestamp(r['expiry_date']).strftime('%y%m%d')}{r['pc']}{r['strike']*1000:08.0f}"
        try:
            prices = api.option_prices(symbol)
        except:
            import traceback
            try:
                traceback.print_last()
            except ValueError:
                print('Error with Traceback')
            return r
        prices = prices[(prices['date'].dt.date > r['date_time'].date())]
        if prices.empty:
            return r

        if r['side'] < 0:
            r['best_open'] = prices.iloc[0]['bid_high']
            b_id = prices.iloc[1:]['ask_low'].idxmin()
            r['best_close'] = prices.loc[b_id]['ask_low']
            r['max_profit'] = r['price'] - r['best_close']
            w_id = prices['bid_high'].idxmax()
            r['worst_close'] = prices.loc[w_id]['bid_high']
            r['min_profit'] = r['price'] - r['worst_close']
        elif r['side'] > 0:
            r['best_open'] = prices.iloc[0]['ask_low']
            b_id = prices.iloc[1:]['bid_high'].idxmax()
            r['best_close'] = prices.loc[b_id]['bid_high']
            r['max_profit'] = r['best_close'] - r['price']
            w_id = prices['ask_low'].idxmin()
            r['worst_close'] = prices.loc[w_id]['ask_low']
            r['min_profit'] = r['worst_close'] - r['price']
        else:
            return r

        r['best_close_date'] = prices.loc[b_id]['date']
        r['max_profit_rate'] = r['max_profit']/r['price']
        r['worst_close_date'] = prices.loc[w_id]['date']
        r['worst_profit_rate'] = r['min_profit']/r['price']
        return r

    #tqdm.pandas(desc=f'{day_id+1}/{len(all_days)} {day.isoformat()}', leave=False)
    #all_dfs.append(trades_df_c.progress_apply(check_results, axis=1))

    for i,r in tqdm(trades_df_c.iterrows(), desc=f'{day_id+1}/{len(all_days)} {day.isoformat()}', leave=False, total=len(trades_df_c)):
        if r['option'] in all_dfs:
            continue
        try:
            prices = api.option_prices(r['option'])
        except:
            print(f'Exception with {r["option"]}')
            continue
        prices.insert(0, 'option', r['option'])
        all_dfs[r['option']] = prices

    all_dfs.append(trades_df_c)

#dfs = pd.concat(all_dfs, axis=0)

1/121 2022-09-28:  11%|█████████████████████████████████████▎                                                                                                                                                                                                                                                                                                          | 15/135 [00:05<00:54,  2.22it/s]

Exception with MAXR221021P00017500


1/121 2022-09-28:  19%|████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                               | 26/135 [00:09<00:32,  3.37it/s]

In [ ]:
df = pd.concat(all_dfs.values(), axis=0, ignore_index=True)

In [ ]:
df = df.sort(by=['date', 'option'])

In [ ]:
pd.read_parquet('../data/history.parquet')

In [ ]:
# TODO Expired Only
#dfs[(dfs['side']>0) & (dfs['best_open']>=dfs['price']) & (dfs['expiry_date']<=np.datetime64('2023-03-18')) & (dfs['max_profit_rate']>=0.2)]

In [ ]:
#119/181, 62/181